# 1 Examining Returns Distribution Data for Stock Data

The basis for the use of heteroskedastic models to forecast volatility is the idea that the homoskedastic assumptions in Black-Scholes Formula derivation is inaccurate.

To preliminarily check whether this is the case, I will assess whether the distribution of returns of underlying securities exhibits homoskedastic or heteroskedastic behavior with significance testing.

## 1.1 Preparing Data

In [3]:
from pandas_datareader import data as pdr
import yfinance as yf

# Downloading data for Microsoft and Apple

yf.pdr_override()

data = pdr.get_data_yahoo("msft aapl", start="2022-01-01", end="2022-12-31")
display(data.head())

[*********************100%%**********************]  2 of 2 completed


Price        Adj Close                   Close                    High  \
Ticker            AAPL        MSFT        AAPL        MSFT        AAPL   
Date                                                                     
2022-01-03  179.724548  328.120850  182.009995  334.750000  182.880005   
2022-01-04  177.443558  322.494537  179.699997  329.010010  182.940002   
2022-01-05  172.723572  310.114624  174.919998  316.380005  180.169998   
2022-01-06  169.840256  307.664215  172.000000  313.880005  175.300003   
2022-01-07  170.008118  307.821014  172.169998  314.040009  174.139999   

Price                          Low                    Open              \
Ticker            MSFT        AAPL        MSFT        AAPL        MSFT   
Date                                                                     
2022-01-03  338.000000  177.710007  329.779999  177.830002  335.350006   
2022-01-04  335.200012  179.119995  326.119995  182.630005  334.829987   
2022-01-05  326.070007  174.639999  315.980011  179.610001  325.859985   
2022-01-06  318.700012  171.639999  311.489990  172.699997  313.149994   
2022-01-07  316.500000  171.029999  310.089996  172.889999  314.149994   

Price          Volume            
Ticker           AAPL      MSFT  
Date                             
2022-01-03  104487900  28865100  
2022-01-04   99310400  32674300  
2022-01-05   94537600  40054300  
2022-01-06   96904000  39646100  
2022-01-07   86709100  32720000

In [4]:
import pandas as pd

# Downloading list of S&P 500 tickers

tickers = pd.read_html(
    'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
print(tickers.head())

  Symbol     Security             GICS Sector               GICS Sub-Industry  \
0    MMM           3M             Industrials        Industrial Conglomerates   
1    AOS  A. O. Smith             Industrials               Building Products   
2    ABT       Abbott             Health Care           Health Care Equipment   
3   ABBV       AbbVie             Health Care                   Biotechnology   
4    ACN    Accenture  Information Technology  IT Consulting & Other Services   

     Headquarters Location  Date added      CIK      Founded  
0    Saint Paul, Minnesota  1957-03-04    66740         1902  
1     Milwaukee, Wisconsin  2017-07-26    91142         1916  
2  North Chicago, Illinois  1957-03-04     1800         1888  
3  North Chicago, Illinois  2012-12-31  1551152  2013 (1888)  
4          Dublin, Ireland  2011-07-06  1467373         1989  


In [5]:
# Downloading data for S&P 500 in 2016
data = pdr.get_data_yahoo(tickers.Symbol.to_list(), start="2016-01-01", end="2016-12-31")
data.head()

[*********************100%%**********************]  503 of 503 completed

21 Failed downloads:
['CTVA', 'GEHC', 'ABNB', 'VLTO', 'IR', 'SOLV', 'CARR', 'KVUE', 'CEG', 'FOX', 'GEV', 'VICI', 'INVH', 'MRNA', 'DAY', 'DOW', 'OTIS', 'UBER', 'FOXA']: Exception("%ticker%: Data doesn't exist for startDate = 1451624400, endDate = 1483160400")
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2016-01-01 -> 2016-12-31)')
['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')


Price       Adj Close                                                   \
Ticker              A        AAL       AAPL       ABBV ABNB        ABT   
Date                                                                     
2016-01-04  38.072586  39.095455  23.946920  40.018429  NaN  36.597130   
2016-01-05  37.941586  38.722763  23.346823  39.851707  NaN  36.588600   
2016-01-06  38.110008  39.401260  22.889931  39.858658  NaN  36.281715   
2016-01-07  36.491276  38.655861  21.923874  39.740566  NaN  35.412182   
2016-01-08  36.107670  38.579411  22.039804  38.656925  NaN  34.670513   

Price                                                   ...    Volume  \
Ticker           ACGL        ACN       ADBE        ADI  ...       WTW   
Date                                                    ...             
2016-01-04  22.950001  88.932762  91.970001  45.964718  ...  22046981   
2016-01-05  23.033333  89.395638  92.339996  45.626980  ...   2163600   
2016-01-06  23.070000  89.220963  91.019997  43.685059  ...         0   
2016-01-07  23.046667  86.600929  89.110001  42.570549  ...   2489500   
2016-01-08  22.806667  85.762512  87.849998  42.199051  ...   2006300   

Price                                                                        \
Ticker           WY     WYNN      XEL       XOM      XYL       YUM      ZBH   
Date                                                                          
2016-01-04  3556800  4249800  2819300  20400100  1353400   4821623  2104084   
2016-01-05  2722800  4286100  2141200  11993500  1075400   5273420  2294737   
2016-01-06  4843200  3316100  5191900  18826900  1866000   5049330  2751233   
2016-01-07  4917400  5525600  5212200  21263800  1555400  11005453  1829589   
2016-01-08  5022600  5740000  3005400  19033600  1471500   5996184  1670969   

Price                         
Ticker         ZBRA      ZTS  
Date                          
2016-01-04   656200  2871700  
2016-01-05   611300  3116700  
2016-01-06   881900  4670500  
2016-01-07  1210600  4324400  
2016-01-08  1034700  4946100  

[5 rows x 3018 columns]

In [6]:
import numpy as np

# Finding log returns for the adjusted close data

returns = np.log1p(data["Adj Close"].pct_change())
for name in returns.columns:
    data[("Daily Log Returns",name)] = returns[name]
returns.head()

C:\Users\Tayo\AppData\Local\Temp\ipykernel_63100\304225854.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[("Daily Log Returns",name)] = returns[name]


Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-05,-0.003447,-0.009579,-0.025379,-0.004175,NaN,-0.000233,0.003624,0.005191,0.004015,-0.007375,...,0.006856,0.002344,0.018727,0.010034,0.008484,-0.000278,-0.002496,0.020609,-0.025437,0.015533
2016-01-06,0.004429,0.017370,-0.019764,0.000174,NaN,-0.008423,0.001591,-0.001956,-0.014398,-0.043493,...,0.000000,-0.022321,-0.053195,0.010483,-0.008355,-0.012554,-0.007106,0.004321,-0.040777,0.000208
2016-01-07,-0.043404,-0.019099,-0.043121,-0.002967,NaN,-0.024258,-0.001012,-0.029805,-0.021208,-0.025843,...,-0.090514,-0.037635,-0.098793,0.003835,-0.016136,-0.026167,-0.034712,-0.023261,-0.046375,-0.030876
2016-01-08,-0.010568,-0.001980,0.005274,-0.027647,NaN,-0.021166,-0.010468,-0.009729,-0.014241,-0.008765,...,0.014424,-0.009276,-0.041556,-0.010995,-0.020409,-0.009556,-0.013701,-0.004226,-0.002697,-0.014713


In [7]:
# Drop symbols with less than 200 datapoints of returns

returns = returns.dropna(axis=1, thresh=len(returns)-200)
returns.tail()

Ticker,A,AAL,AAPL,ABBV,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2016-12-23,0.004991,-0.004322,0.001976,0.010968,0.003389,0.001835,-0.002635,0.002861,0.005295,-0.000664,...,0.013261,-0.001634,0.001019,0.000000,-0.001762,-0.002350,-0.003754,0.007357,0.002897,0.009716
2016-12-27,0.006688,0.002678,0.006331,0.002563,0.004674,-0.006324,0.000596,-0.000381,0.006345,0.011013,...,-0.001688,-0.004590,-0.001133,0.001474,0.000441,-0.002945,0.003442,0.007982,0.007264,-0.001116
2016-12-28,-0.017132,-0.019527,-0.004273,-0.003687,-0.009893,-0.008107,-0.008029,-0.011593,-0.010144,-0.006814,...,-0.011536,-0.017905,-0.013233,-0.010861,-0.004971,-0.018255,-0.004853,0.001554,-0.016681,-0.005226
2016-12-29,0.001140,-0.008850,-0.000257,0.007360,0.002352,0.001046,0.003424,-0.000868,0.000272,0.000441,...,-0.001717,0.015273,0.001033,0.016004,0.000553,-0.003410,-0.001413,0.004260,0.003847,0.003363
2016-12-30,-0.001754,-0.011922,-0.007826,-0.001755,0.002607,0.002320,0.001025,-0.007066,-0.013133,0.006373,...,0.000900,-0.008604,-0.007600,-0.005879,-0.000996,-0.005036,-0.004726,-0.002902,-0.002097,-0.001680


In [8]:
data.head()

Price       Adj Close                                                   \
Ticker              A        AAL       AAPL       ABBV ABNB        ABT   
Date                                                                     
2016-01-04  38.072586  39.095455  23.946920  40.018429  NaN  36.597130   
2016-01-05  37.941586  38.722763  23.346823  39.851707  NaN  36.588600   
2016-01-06  38.110008  39.401260  22.889931  39.858658  NaN  36.281715   
2016-01-07  36.491276  38.655861  21.923874  39.740566  NaN  35.412182   
2016-01-08  36.107670  38.579411  22.039804  38.656925  NaN  34.670513   

Price                                                   ... Daily Log Returns  \
Ticker           ACGL        ACN       ADBE        ADI  ...               WTW   
Date                                                    ...                     
2016-01-04  22.950001  88.932762  91.970001  45.964718  ...               NaN   
2016-01-05  23.033333  89.395638  92.339996  45.626980  ...          0.006856   
2016-01-06  23.070000  89.220963  91.019997  43.685059  ...          0.000000   
2016-01-07  23.046667  86.600929  89.110001  42.570549  ...         -0.090514   
2016-01-08  22.806667  85.762512  87.849998  42.199051  ...          0.014424   

Price                                                                   \
Ticker            WY      WYNN       XEL       XOM       XYL       YUM   
Date                                                                     
2016-01-04       NaN       NaN       NaN       NaN       NaN       NaN   
2016-01-05  0.002344  0.018727  0.010034  0.008484 -0.000278 -0.002496   
2016-01-06 -0.022321 -0.053195  0.010483 -0.008355 -0.012554 -0.007106   
2016-01-07 -0.037635 -0.098793  0.003835 -0.016136 -0.026167 -0.034712   
2016-01-08 -0.009276 -0.041556 -0.010995 -0.020409 -0.009556 -0.013701   

Price                                     
Ticker           ZBH      ZBRA       ZTS  
Date                                      
2016-01-04       NaN       NaN       NaN  
2016-01-05  0.020609 -0.025437  0.015533  
2016-01-06  0.004321 -0.040777  0.000208  
2016-01-07 -0.023261 -0.046375 -0.030876  
2016-01-08 -0.004226 -0.002697 -0.014713  

[5 rows x 3521 columns]

## 1.2 Measuring Distributions Data

Constructing tables to describe the returns distribution for each security

Skewness measures the symmetry of the returns distribution - how positively or negatively skewed it is
Kurtosis indicates how much of the probability density resides in the tails of the distribution
K-Squared P-Value measures the likelihood that the distribution of returns over the period measured is normal 
- A low P-Value (less than 0.05) meets the 5% significance threshold to reject the null hypothesis and assume that the distribution of the returns for each security over the period measured is not normal 
- Returns could be instantaneously normal, with the same mean but different volatility (standard deviation) at each timestep, this would explain the reasonable skewness measurements (in [-1, 1]) but large kurtosis measurements (outside [-2, 2])

In [9]:
# Constructing distribution dataframes, and dropping nan values

import scipy.stats as stats

symbols = returns.columns
sample_stats = pd.DataFrame({"Observations": (len(returns[symbol]) for symbol in symbols),
                      "Mean Return": list(np.mean(returns, axis=0)),
                      "Standard Deviation": list(np.std(returns, axis=0)),
                      "Skewness": list(stats.skew(returns, axis=0, nan_policy='omit')),
                      "Kurtosis": list(stats.kurtosis(returns, axis=0, nan_policy='omit')),
                      "K-Squared P-Value": list(stats.normaltest(returns, axis=0, nan_policy='omit').pvalue)},
                      index = symbols)
sample_stats.head()

,Observations,Mean Return,Standard Deviation,Skewness,Kurtosis,K-Squared P-Value
Ticker,,,,,,
A,252,0.000494,0.014772,-0.822092,1.961764,4.789181e-09
AAL,252,0.000571,0.023052,-0.379117,3.623917,6.492817e-08
AAPL,252,0.000465,0.014723,-0.320055,4.301027,1.749258e-08
ABBV,252,0.000483,0.017080,0.016758,3.090483,7.970641e-06
ABT,252,-0.000344,0.015094,-1.907169,9.541369,2.817993e-29


In [10]:
measured = sum(sample_stats["K-Squared P-Value"] != np.nan)
non_normal = len(sample_stats[sample_stats["K-Squared P-Value"] < 0.05])
print(f"Of {measured} S&P 500 stocks with a measurable K-Squared Statistic, \
\n{non_normal} have sufficiently low P-Value to reject the null hypothesis\
\nthat their returns are normally distributed")

Of 480 S&P 500 stocks with a measurable K-Squared Statistic, 
474 have sufficiently low P-Value to reject the null hypothesis
that their returns are normally distributed


*Function to find returns distribution for stock data, and test hypothesis of normally distributed returns with constant variance*

In [11]:
def returns_distribution(tickers: list[str], start: str, end:str, filename=None, header=None):
    """
    Finds returns distribution for stock data, and tests hypothesis of normally distributed returns
    with constant variance
    :param tickers: The ticker symbol for data to be downloaded
    :param start: The start date for downloaded stock data
    :param end: The end date for downloaded stock data
    :param filename: The name of the file for the data to be uploaded to (optional)
    :param header: The header configuration for the file (optional)
    :return: Dataframe with the number of observations, mean return, standard deviation of returns,
    skewness of returns, kurtosis of returns, and k-squared test p-value of the returns
    """
    data = yf.download(tickers, start=start, end=end)
    returns = np.log1p(data["Close"].pct_change())
    returns = returns.dropna(axis=1, thresh=len(returns)-200)
    
    symbols = returns.columns
    sample_stats = pd.DataFrame({"Observations": (len(returns[symbol]) for symbol in symbols),
                      "Mean Return": list(np.mean(returns, axis=0)),
                      "Standard Deviation": list(np.std(returns, axis=0)),
                      "Skewness": list(stats.skew(returns, axis=0, nan_policy='omit')),
                      "Kurtosis": list(stats.kurtosis(returns, axis=0, nan_policy='omit')),
                      "K-Squared P-Value": list(stats.normaltest(returns, axis=0, nan_policy='omit').pvalue)},
                      index = symbols)
    
    if filename is not None:
        sample_stats.to_csv(filename, header=header)
    
    return sample_stats

In [12]:
import pandas as pd

tickers = pd.read_html(
    'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

symbols = tickers.Symbol.to_list()
ss = returns_distribution(symbols, "2022-01-01", "2022-12-31", filename="rd-2022.csv")
ss.head()

[*********************100%%**********************]  503 of 503 completed

6 Failed downloads:
['VLTO', 'SOLV', 'KVUE', 'GEV']: Exception("%ticker%: Data doesn't exist for startDate = 1641013200, endDate = 1672462800")
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2022-01-01 -> 2022-12-31)')
['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')


,Observations,Mean Return,Standard Deviation,Skewness,Kurtosis,K-Squared P-Value
Ticker,,,,,,
A,251,-0.000179,0.022148,0.308525,0.488630,3.953602e-02
AAL,251,-0.001552,0.035410,-0.030688,0.572806,2.124210e-01
AAPL,251,-0.001348,0.022387,0.231176,0.865158,2.134347e-02
ABBV,251,0.000707,0.014295,-0.882248,2.856754,3.825678e-11
ABNB,251,-0.002812,0.038346,-0.326138,0.434121,3.813268e-02


In [27]:
ss.head().to_latex(float_format="%.3f")

C:\Users\Tayo\AppData\Local\Temp\ipykernel_63100\132591234.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  ss.head().to_latex(float_format="%.3f")


'\\begin{tabular}{lrrrrrr}\n\\toprule\n{} &  Observations &  Mean Return &  Standard Deviation &  Skewness &  Kurtosis &  K-Squared P-Value \\\\\nTicker &               &              &                     &           &           &                    \\\\\n\\midrule\nA      &           251 &       -0.000 &               0.022 &     0.309 &     0.489 &              0.040 \\\\\nAAL    &           251 &       -0.002 &               0.035 &    -0.031 &     0.573 &              0.212 \\\\\nAAPL   &           251 &       -0.001 &               0.022 &     0.231 &     0.865 &              0.021 \\\\\nABBV   &           251 &        0.001 &               0.014 &    -0.882 &     2.857 &              0.000 \\\\\nABNB   &           251 &       -0.003 &               0.038 &    -0.326 &     0.434 &              0.038 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [14]:
measured = non_normal = 0
for year in range(2000, 2023):
    start = f"{year}-01-01"
    end = f"{year}-12-31"
    rd = returns_distribution(symbols, start, end, filename=f"data/rd-{year}.csv")
    non_normal += sum(sample_stats["K-Squared P-Value"] < 0.05)
    measured += len(sample_stats)

[*********************100%%**********************]  503 of 503 completed

138 Failed downloads:
['KDP', 'NWSA', 'CTVA', 'PM', 'MSCI', 'GEHC', 'WYNN', 'PFG', 'KVUE', 'TDG', 'FANG', 'CTLT', 'WRK', 'CME', 'SMCI', 'CE', 'KEYS', 'ULTA', 'WBD', 'MPWR', 'AXON', 'CZR', 'BR', 'NFLX', 'IQV', 'NCLH', 'DLR', 'MDLZ', 'V', 'TSLA', 'CDW', 'ALGN', 'CBOE', 'CBRE', 'HLT', 'LYV', 'CHTR', 'CPAY', 'ABNB', 'CMG', 'TRGP', 'NDAQ', 'AAL', 'LKQ', 'PYPL', 'PAYC', 'BG', 'WTW', 'IR', 'STX', 'QRVO', 'VLTO', 'PSX', 'NRG', 'ELV', 'EXPE', 'ANET', 'DAL', 'META', 'BX', 'APTV', 'CEG', 'CARR', 'KHC', 'SOLV', 'GOOGL', 'FOX', 'ICE', 'PODD', 'AVGO', 'GEV', 'MA', 'LDOS', 'EPAM', 'ABBV', 'TMUS', 'GPN', 'AIZ', 'VICI', 'GNRC', 'ALLE', 'LULU', 'FTV', 'AWK', 'INVH', 'PRU', 'ZTS', 'HWM', 'MPC', 'CFG', 'MRNA', 'NOW', 'ZBH', 'ACN', 'DFS', 'PANW', 'BLDR', 'GM', 'DAY', 'AMP', 'ENPH', 'VST', 'VRSK', 'PARA', 'ETSY', 'MOH', 'DXCM', 'LVS', 'LW', 'FTNT', 'DOW', 'DG', 'HPE', 'AMCR', 'TEL', 'GOOG', 'KMI', 'UAL', 'CNC', 'NWS', 'HCA', 'DPZ', 'C

In [15]:
print(f"From 2000 to 2023, of {measured} measured year long stock returns behaviour, \
\n{non_normal} showed behaviour sufficient to reject the null-hypothesis that they\
\nare normally distributed with constant variance, based on the K-Squared Test")

From 2000 to 2022, of 11040 measured year long stock returns behaviour, 
10902 showed behaviour sufficient to reject the null-hypothesis that they
are normally distributed with constant variance, based on the K-Squared Test


From this analysis, we can see that generally, the securities in the S&P 500 do not appear to have returns that exhibit normal distribution with constant variance over a year long measurement basis.

## 1.3 Measuring Autocorrelation Data

Constructing tables to describe the autocorrelative behaviour of the returns distribution for each security

Distributions that exhibit autocorrelative behaviour, suggest that the use of autoregressive models to predict future measurements is sound
This is pertinent as it provides motivation for the use of autoregressive models like ARCH, and ARIMA to forecast behaviour

Autocorrelation for squared and absolute returns are also measured, this is because autocorrelation between these measures suggests that the volatility of the distribution may follow some sort of process that leads to the volatility of measures across some time lag being correlated.

In [28]:
from statsmodels.tsa.stattools import acf, pacf

In [29]:
# Autocorrelation stats for stocks: autocorrelation, ljung-box q-statistic, p-value
def ac_format(ac):
    """
    Filters acf function to remove autocorrelation value for 0 lag to make array same dimension
    """
    lags = ac[0][1:6]
    rest = ac[1:3]
    return [lags] + list(rest)

full_r_ac = np.array(list(ac_format(acf(returns[symbol].dropna(), nlags=5, qstat=True)) for symbol in returns.columns))
full_sqr_r_ac = np.array(list(ac_format(acf(returns[symbol].dropna()**2, nlags=5, qstat=True)) for symbol in returns.columns))
full_abs_r_ac = np.array(list(ac_format(acf(abs(returns[symbol].dropna()), nlags=5, qstat=True)) for symbol in returns.columns))
r_pac = np.array(list(pacf(returns[symbol].dropna(), nlags=5)[1:6] for symbol in returns.columns)).T
sqr_r_pac = np.array(list(pacf(returns[symbol].dropna()**2, nlags=5)[1:6] for symbol in returns.columns)).T
abs_r_pac = np.array(list(pacf(abs(returns[symbol].dropna()), nlags=5)[1:6] for symbol in returns.columns)).T

In [30]:
# Slicing to get autocorrelations, q-statistic, and p-values exclusively
r_ac = full_r_ac[:,0].T
r_ldq = full_r_ac[:,1].T
r_pval = full_r_ac[:,2].T
sqr_r_ac = full_sqr_r_ac[:,0].T
sqr_r_ldq = full_sqr_r_ac[:,1].T
sqr_r_pval = full_sqr_r_ac[:,2].T
abs_r_ac = full_abs_r_ac[:,0].T
abs_r_ldq = full_abs_r_ac[:,1].T
abs_r_pval = full_abs_r_ac[:,2].T

In [31]:
# P-Value data-frame for different time lags
p = pd.DataFrame(
    {
    **{("Returns Autocorrelation P Value", f"Lag {l+1}"): r_pval[l] for l in range(5)},
    **{("Squared Returns Autocorrelation P Value", f"Lag {l+1}"): sqr_r_pval[l] for l in range(5)},
    **{("Absolute Returns Autocorrelation P Value", f"Lag {l+1}"): abs_r_pval[l] for l in range(5)}
    }, index = returns.columns
)
p.head()

Returns Autocorrelation P Value                                \
                                 Lag 1     Lag 2     Lag 3     Lag 4   
Ticker                                                                 
A                             0.130917  0.268308  0.367427  0.530028   
AAL                           0.357316  0.650907  0.720298  0.669916   
AAPL                          0.233451  0.484040  0.358536  0.416643   
ABBV                          0.164142  0.165964  0.064057  0.108294   
ABT                           0.333670  0.521988  0.464739  0.626952   

                 Squared Returns Autocorrelation P Value                \
           Lag 5                                   Lag 1         Lag 2   
Ticker                                                                   
A       0.385178                                0.050568  1.255183e-01   
AAL     0.581121                                0.010125  5.751231e-03   
AAPL    0.553333                                0.544744  3.409326e-01   
ABBV    0.081168                                0.000177  5.163733e-11   
ABT     0.758816                                0.110812  2.722011e-01   

                                                  \
               Lag 3         Lag 4         Lag 5   
Ticker                                             
A       1.702689e-01  2.784990e-01  2.353515e-01   
AAL     1.273568e-02  2.384105e-02  4.631155e-02   
AAPL    9.611721e-02  1.554885e-01  2.137251e-01   
ABBV    8.994531e-11  2.861143e-10  1.086038e-09   
ABT     4.408890e-01  5.868684e-01  7.057776e-01   

       Absolute Returns Autocorrelation P Value                              \
                                          Lag 1         Lag 2         Lag 3   
Ticker                                                                        
A                                      0.047789  1.401945e-01  2.385649e-01   
AAL                                    0.051397  2.180934e-02  2.957707e-02   
AAPL                                   0.059914  7.846372e-03  7.139986e-03   
ABBV                                   0.004959  6.047478e-11  4.925365e-11   
ABT                                    0.001523  6.260233e-03  1.685287e-02   

                                    
               Lag 4         Lag 5  
Ticker                              
A       2.641755e-01  8.410271e-02  
AAL     5.376991e-02  7.861857e-02  
AAPL    3.354373e-03  4.552854e-03  
ABBV    3.712787e-11  7.741084e-11  
ABT     3.587046e-02  6.647527e-02

In [32]:
#Ljung-Box Q-Statistic Dataframe for different time lags
ldq = pd.DataFrame(
    {
    **{("Returns Ljung-Box Q Statistic", f"Lag {l+1}"): r_ldq[l] for l in range(5)},
    **{("Squared Returns Ljung-Box Q Statistic", f"Lag {l+1}"): sqr_r_ldq[l] for l in range(5)},
    **{("Absolute Returns Ljung-Box Q Statistic", f"Lag {l+1}"): abs_r_ldq[l] for l in range(5)}
    }, index = returns.columns
)
ldq.head()

Returns Ljung-Box Q Statistic                                          \
                               Lag 1     Lag 2     Lag 3     Lag 4     Lag 5   
Ticker                                                                         
A                           2.281597  2.631236  3.161238  3.168533  5.258287   
AAL                         0.847304  0.858776  1.337290  2.359735  3.782601   
AAPL                        1.419711  1.451176  3.222774  3.922156  3.972838   
ABBV                        1.935649  3.591972  7.259974  7.578565  9.797914   
ABT                         0.934602  1.300221  2.558910  2.599272  2.616765   

       Squared Returns Ljung-Box Q Statistic                                   \
                                       Lag 1      Lag 2      Lag 3      Lag 4   
Ticker                                                                          
A                                   3.822534   4.150607   5.020959   5.086877   
AAL                                 6.612842  10.316683  10.820755  11.255174   
AAPL                                0.366818   2.152141   6.341741   6.651550   
ABBV                               14.055629  47.373553  49.758280  50.483981   
ABT                                 2.542601   2.602428   2.696109   2.828819   

                  Absolute Returns Ljung-Box Q Statistic             \
            Lag 5                                  Lag 1      Lag 2   
Ticker                                                                
A        6.807543                               3.917374   3.929448   
AAL     11.268346                               3.795275   7.650834   
AAPL     7.094245                               3.539764   9.695408   
ABBV    50.517146                               7.894188  47.057589   
ABT      2.962433                              10.050469  10.147076   

                                         
            Lag 3      Lag 4      Lag 5  
Ticker                                   
A        4.221029   5.233410   9.702962  
AAL      8.978577   9.311412   9.883018  
AAPL    12.072169  15.762912  16.972184  
ABBV    50.986180  54.723385  56.102485  
ABT     10.211294  10.286369  10.327292

In [33]:
#Autocorrelation Dataframe for different time lags
corr = pd.DataFrame(
    {
    **{("Returns Autocorrelation", f"Lag {l+1}"): r_ac[l] for l in range(5)},
    **{("Square Returns Autocorrelation", f"Lag {l+1}"): sqr_r_ac[l] for l in range(5)},
    **{("Absolute Returns Autocorrelation", f"Lag {l+1}"): abs_r_ac[l] for l in range(5)},
    **{("Returns Partial Autocorrelation", f"Lag {l+1}"): r_pac[l] for l in range(5)},
    **{("Square Returns Partial Autocorrelation", f"Lag {l+1}"): sqr_r_pac[l] for l in range(5)},
    **{("Absolute Returns Partial Autocorrelation", f"Lag {l+1}"): abs_r_pac[l] for l in range(5)}
    }, index = returns.columns)
corr.head()

Returns Autocorrelation                                          \
                         Lag 1     Lag 2     Lag 3     Lag 4     Lag 5   
Ticker                                                                   
A                    -0.094775 -0.037026  0.045495 -0.005327 -0.089974   
AAL                   0.057755  0.006707  0.043229 -0.063063 -0.074242   
AAPL                  0.074761 -0.011108 -0.083179 -0.052156  0.014012   
ABBV                 -0.087294 -0.080589  0.119686 -0.035202 -0.092722   
ABT                  -0.060658 -0.037863  0.070111  0.012530  0.008232   

       Square Returns Autocorrelation                                          \
                                Lag 1     Lag 2     Lag 3     Lag 4     Lag 5   
Ticker                                                                          
A                            0.122673 -0.035866 -0.058301  0.016012  0.081643   
AAL                          0.161349  0.120512  0.044369 -0.041106 -0.007143   
AAPL                         0.038001  0.083668  0.127913  0.034713  0.041412   
ABBV                         0.235233  0.361444  0.096505  0.053129  0.011335   
ABT                          0.100049 -0.015316 -0.019127 -0.022720 -0.022751   

        ... Square Returns Partial Autocorrelation                      \
        ...                                  Lag 1     Lag 2     Lag 3   
Ticker  ...                                                              
A       ...                               0.123164 -0.052114 -0.048773   
AAL     ...                               0.161995  0.097804  0.011724   
AAPL    ...                               0.038153  0.083006  0.124380   
ABBV    ...                               0.236174  0.326798 -0.044390   
ABT     ...                               0.100449 -0.025789 -0.015342   

                           Absolute Returns Partial Autocorrelation            \
           Lag 4     Lag 5                                    Lag 1     Lag 2   
Ticker                                                                          
A       0.028698  0.074726                                 0.124682 -0.008746   
AAL    -0.064145  0.002130                                 0.122724  0.110546   
AAPL    0.021206  0.020755                                 0.118521  0.144592   
ABBV   -0.088049 -0.001997                                 0.176995  0.375453   
ABT    -0.020155 -0.019734                                 0.199710 -0.061975   

                                      
           Lag 3     Lag 4     Lag 5  
Ticker                                
A      -0.034474  0.073618  0.119729  
AAL     0.047059 -0.064877  0.046591  
AAPL    0.066810  0.086687  0.028944  
ABBV    0.018319 -0.049565  0.014627  
ABT     0.034050 -0.030310  0.025749  

[5 rows x 30 columns]

In [45]:
# Number of significant p values in observations
sig_p = np.sum((p < 0.05).any(axis=1))
sig_p

329

In [50]:
def returns_autocorrelation(tickers: list[str], start:str, end:str, filename=None, header=None):
    """
    Returns three dataframes of autocorrelation, ljung-box q statistic, and p-value for returns
    of stocks in the list of tickers
    :param tickers: list of tickers
    :param start: start date to download data from
    :param end: end date to download data from
    :param filename: name of file to upload data to (optional)
    :param header: header configuration of the file (optional)
    :return: Three dataframes, autocorrelation, ljung-box q statistic, and p-value
    """
    data = yf.download(tickers, start=start, end=end)
    returns = np.log1p(data["Adj Close"].pct_change())
    returns = returns.dropna(axis=1, thresh=len(returns)-200)

    def ac_format(ac):
        lags = ac[0][1:6]
        rest = ac[1:3]
        return [lags] + list(rest)

    full_r_ac = np.array(list(ac_format(acf(returns[symbol].dropna(), nlags=5, qstat=True)) for symbol in returns.columns))
    full_sqr_r_ac = np.array(list(ac_format(acf(returns[symbol].dropna()**2, nlags=5, qstat=True)) for symbol in returns.columns))
    full_abs_r_ac = np.array(list(ac_format(acf(abs(returns[symbol].dropna()), nlags=5, qstat=True)) for symbol in returns.columns))
    r_pac = np.array(list(pacf(returns[symbol].dropna(), nlags=5)[1:6] for symbol in returns.columns)).T
    sqr_r_pac = np.array(list(pacf(returns[symbol].dropna()**2, nlags=5)[1:6] for symbol in returns.columns)).T
    abs_r_pac = np.array(list(pacf(abs(returns[symbol].dropna()), nlags=5)[1:6] for symbol in returns.columns)).T
    
    r_ac = full_r_ac[:,0].T
    r_ldq = full_r_ac[:,1].T
    r_pval = full_r_ac[:,2].T
    sqr_r_ac = full_sqr_r_ac[:,0].T
    sqr_r_ldq = full_sqr_r_ac[:,1].T
    sqr_r_pval = full_sqr_r_ac[:,2].T
    abs_r_ac = full_abs_r_ac[:,0].T
    abs_r_ldq = full_abs_r_ac[:,1].T
    abs_r_pval = full_abs_r_ac[:,2].T

    symbols = returns.columns
    corr = pd.DataFrame(
    {
    **{("Returns Autocorrelation", f"Lag {l+1}"): r_ac[l] for l in range(5)},
    **{("Square Returns Autocorrelation", f"Lag {l+1}"): sqr_r_ac[l] for l in range(5)},
    **{("Absolute Returns Autocorrelation", f"Lag {l+1}"): abs_r_ac[l] for l in range(5)},
    **{("Returns Partial Autocorrelation", f"Lag {l+1}"): r_pac[l] for l in range(5)},
    **{("Square Returns Partial Autocorrelation", f"Lag {l+1}"): sqr_r_pac[l] for l in range(5)},
    **{("Absolute Returns Partial Autocorrelation", f"Lag {l+1}"): abs_r_pac[l] for l in range(5)}
    }, index = returns.columns)

    ldq = pd.DataFrame(
    {
    **{("Returns Ljung-Box Q Statistic", f"Lag {l+1}"): r_ldq[l] for l in range(5)},
    **{("Squared Returns Ljung-Box Q Statistic", f"Lag {l+1}"): sqr_r_ldq[l] for l in range(5)},
    **{("Absolute Returns Ljung-Box Q Statistic", f"Lag {l+1}"): abs_r_ldq[l] for l in range(5)}
    }, index = returns.columns)

    p = pd.DataFrame(
    {
    **{("Returns Autocorrelation P Value", f"Lag {l+1}"): r_pval[l] for l in range(5)},
    **{("Squared Returns Autocorrelation P Value", f"Lag {l+1}"): sqr_r_pval[l] for l in range(5)},
    **{("Absolute Returns Autocorrelation P Value", f"Lag {l+1}"): abs_r_pval[l] for l in range(5)}
    }, index = returns.columns)

    if filename is not None:
        corr.to_csv("data/corr-"+filename, header=header)
        p.to_csv("data/p-"+filename, header=header)

    return corr, ldq, p

In [51]:
for year in range(2000, 2023):
    corr,_,p = returns_autocorrelation(list(symbols), f"{year}-01-01", f"{year}-12-31",\
        filename=f"rcd-{year}.csv")
    if year == 2000:
        measured_p = np.sum(p != np.nan, axis=0)
        sig_p = np.sum(p < 0.05, axis=0)
        any_sig_p = np.sum((p < 0.05).any(axis=1))
    else:
        measured_p += np.sum(p != np.nan, axis=0)
        sig_p += np.sum(p < 0.05, axis=0)
        any_sig_p += np.sum((p < 0.05).any(axis=1))

[*********************100%%**********************]  503 of 503 completed

138 Failed downloads:
['MSCI', 'PM', 'KDP', 'NWSA', 'CTVA', 'GEHC', 'KHC', 'PFG', 'TDG', 'FANG', 'CTLT', 'SMCI', 'WRK', 'CE', 'KEYS', 'ULTA', 'CME', 'AXON', 'CZR', 'NFLX', 'BR', 'MPWR', 'WBD', 'IQV', 'NCLH', 'V', 'MDLZ', 'ALGN', 'CDW', 'TSLA', 'DLR', 'CBOE', 'CBRE', 'HLT', 'CHTR', 'ABNB', 'LYV', 'CPAY', 'CMG', 'TRGP', 'AAL', 'LKQ', 'NDAQ', 'PAYC', 'BG', 'PYPL', 'STX', 'IR', 'WTW', 'VLTO', 'PSX', 'QRVO', 'BX', 'EXPE', 'ELV', 'NRG', 'ANET', 'DAL', 'META', 'APTV', 'WYNN', 'CARR', 'SOLV', 'GOOGL', 'CEG', 'KVUE', 'FOX', 'ICE', 'MA', 'LDOS', 'AVGO', 'PODD', 'GEV', 'EPAM', 'TMUS', 'ABBV', 'GPN', 'AIZ', 'GNRC', 'VICI', 'AWK', 'ALLE', 'LULU', 'FTV', 'INVH', 'HWM', 'ZTS', 'PRU', 'MPC', 'MRNA', 'NOW', 'CFG', 'ZBH', 'DFS', 'PANW', 'ACN', 'BLDR', 'GM', 'ENPH', 'DAY', 'AMP', 'VST', 'VRSK', 'PARA', 'ETSY', 'DXCM', 'MOH', 'LW', 'LVS', 'AMCR', 'GOOG', 'DOW', 'FTNT', 'DG', 'UAL', 'TEL', 'HPE', 'EXR', 'NWS', 'CNC', 'KMI', 'HCA', 'C

In [52]:
measured_p

Returns Autocorrelation P Value           Lag 1    10177
                                          Lag 2    10177
                                          Lag 3    10177
                                          Lag 4    10177
                                          Lag 5    10177
Squared Returns Autocorrelation P Value   Lag 1    10177
                                          Lag 2    10177
                                          Lag 3    10177
                                          Lag 4    10177
                                          Lag 5    10177
Absolute Returns Autocorrelation P Value  Lag 1    10177
                                          Lag 2    10177
                                          Lag 3    10177
                                          Lag 4    10177
                                          Lag 5    10177
dtype: int64

In [53]:
sig_p

Returns Autocorrelation P Value           Lag 1    1541
                                          Lag 2    1669
                                          Lag 3    1674
                                          Lag 4    1692
                                          Lag 5    1735
Squared Returns Autocorrelation P Value   Lag 1    2887
                                          Lag 2    3333
                                          Lag 3    3473
                                          Lag 4    3555
                                          Lag 5    3616
Absolute Returns Autocorrelation P Value  Lag 1    4001
                                          Lag 2    4446
                                          Lag 3    4602
                                          Lag 4    4751
                                          Lag 5    4765
dtype: int64

In [54]:
any_sig_p

6722

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, fixed
from IPython.display import display

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np
def plot_sqr_returns_autocorrelation(ticker, transform, year, lags):
    match transform:
        case "Returns":
            t = lambda x: x
        case "Squared Returns":
            t = lambda x: x**2
        case "Absolute Returns":
            t = lambda x: abs(x)
    start = f"{year}-01-01"
    end = f"{year}-12-31"
    data = yf.download(ticker, start=start, end=end)
    returns = np.log1p(data["Adj Close"].pct_change())
    plot_acf(t(returns).dropna(), lags=lags)

In [ ]:
ticker_text = widgets.Text(
    value="^GSPC",
    placeholder="Ticker Symbol"
)
year_text = widgets.BoundedIntText(
    value=2000,
    min=1970,
    max=2022
)
lags_text = widgets.BoundedIntText(
    value=5,
    min=3,
    max=100
)
transform_select = widgets.Select(options=["Returns", "Squared Returns", "Absolute Returns"])

In [ ]:
_ = interact(plot_sqr_returns_autocorrelation,
            ticker = ticker_text,
            year = year_text,
            lags = lags_text,
            transform = transform_select)

interactive(children=(Text(value='^GSPC', description='ticker', placeholder='Ticker Symbol'), Select(descripti…